In [1]:
from langchain_community.document_loaders import UnstructuredURLLoader
urls = ['https://www.exelatech.com','https://www.exelatech.com/about-us','https://www.exelatech.com/careers']
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()  

In [2]:
data

[Document(metadata={'source': 'https://www.exelatech.com'}, page_content='Driving Growth and Innovation through Business Process Automation\n\nExela Technologies BPA, LLC is a global leader in business process automation (BPA). We help businesses automate their processes, improve efficiency, and reduce costs. Our solutions are used by a wide range of customers, including Fortune 500 companies, government agencies, and healthcare organizations.\n\nWe have a proven track record of success and are well-positioned to continue to grow in the future. We are the recipient of numerous awards for our innovation and customer service.\n\nOur current and emerging award-winning solutions include:\n\nLiquidity Solutions:\n\nAutomate key financial processes for time-saving, accuracy, and risk reduction.\n\nHuman Capital Management:\n\nStreamline HR tasks like payroll and recruitment for cost-saving and improved talent management.\n\nHealthcare Payers and Revenue Cycle Management (RCM):\n\nAutomate cl

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  19


In [4]:
docs[0]

Document(metadata={'source': 'https://www.exelatech.com'}, page_content='Driving Growth and Innovation through Business Process Automation\n\nExela Technologies BPA, LLC is a global leader in business process automation (BPA). We help businesses automate their processes, improve efficiency, and reduce costs. Our solutions are used by a wide range of customers, including Fortune 500 companies, government agencies, and healthcare organizations.\n\nWe have a proven track record of success and are well-positioned to continue to grow in the future. We are the recipient of numerous awards for our innovation and customer service.\n\nOur current and emerging award-winning solutions include:\n\nLiquidity Solutions:\n\nAutomate key financial processes for time-saving, accuracy, and risk reduction.\n\nHuman Capital Management:\n\nStreamline HR tasks like payroll and recruitment for cost-saving and improved talent management.\n\nHealthcare Payers and Revenue Cycle Management (RCM):\n\nAutomate cla

In [5]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(docs, embedding=embedding)


/var/folders/rm/mnlp413s2jbcq_6pl2mg0dlh0000gn/T/ipykernel_1588/443485189.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


vocab.txt: 0.00B [00:00, ?B/s]

In [6]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What kind of services they provide?")


In [7]:
len(retrieved_docs)

6

In [8]:
print(retrieved_docs[1].page_content)

Read more

DrySign Logo

Everyday Signing Made Easy

Secure and legally binding signatures on the go.

Read more

Digital Mailroom Logo

Digital Mailroom

Streamline your mail delivery process.

Read more

EHRS Logo

Revolutionizing Human Resource Management

Powerful HR solutions to accelerate performance and business growth.

Read more

PCH Global

PCH Global

Experience a unified environment between providers, payers, and patients with a centralized access for claims management.

Read more

XME Logo

Driving Market Results

Integrated solutions to execute end-to-end marketing approach.

Read more

XES Logo

Exela Enterprise Solutions

Transform your business with the power of human insight and technology with Exela Enterprise Solutions.

Read more

See all Solutions

XBP

Optimize Accounts Payables and Receivables in One Simple Platform


In [9]:
llm = OpenAI(temperature=0.4, max_tokens=500)

In [10]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [11]:

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "What kind of services they provide?"})
print(response["answer"])